In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
from sklearn.metrics import f1_score
import os

In [7]:
 import pandas as pd
import numpy as np
import tarfile
import io
import os
# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split
from skimage import io
import PIL
import torch
# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from PIL import Image
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, ToPILImage, Normalize, Compose
import torch.nn.functional as F

#from torchvision.transforms import ToTensor, ToPILImage
# PyTorch libraries and modules
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.optim as optim

from timeit import default_timer as timer  

In [9]:
os.chdir(r'D:\NEU Courseworks\CSYE7105 - Parallel Machine Learning & AI\Final-Project\Folder\CNN for Hair Feature Prediction')

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
df_attr = pd.read_csv('list_attr_celeba.csv')

In [14]:
df_attr.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [15]:
import csv
updatedlist=[]
with open("train.csv",newline="") as f:
    reader=csv.reader(f)
    for row in reader: 
        if '1' in row:
            updatedlist.append(row)
            

In [16]:
len(updatedlist)

100868

In [17]:
with open('output_train.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['image_id', 'Black_Hair', 'Blond_Hair','Brown_Hair','Gray_Hair'])
    writer.writerows(updatedlist)

In [18]:
updated_test_list=[]
with open("test.csv",newline="") as f:
    reader=csv.reader(f)
    for row in reader: 
        if '1' in row:
            updated_test_list.append(row)
            

In [20]:
len(updated_test_list)

12113

In [21]:
with open('output_test.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['image_id', 'Black_Hair', 'Blond_Hair','Brown_Hair','Gray_Hair'])
    writer.writerows(updated_test_list)

In [22]:
updated_valid_list=[]
with open("Validation.csv",newline="") as f:
    reader=csv.reader(f)
    for row in reader: 
        if '1' in row:
            updated_valid_list.append(row)

In [23]:
len(updated_valid_list)

12615

In [24]:
with open('output_valid.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['image_id', 'Black_Hair', 'Blond_Hair','Brown_Hair','Gray_Hair'])
    writer.writerows(updated_valid_list)

In [25]:
df_train = pd.read_csv('output_train.csv')

In [26]:
df_train.head()

,image_id,Black_Hair,Blond_Hair,Brown_Hair,Gray_Hair
0,000001.jpg,0,0,1,0
1,000002.jpg,0,0,1,0
2,000006.jpg,0,0,1,0
3,000007.jpg,1,0,0,0
4,000008.jpg,1,0,0,0


In [27]:
training_imgs = ["{}".format(x) for x in list(df_train.image_id)]

In [28]:
training_labels_1 = list(df_train['image_id'])
training_labels_2 = list(df_train['Black_Hair'])
training_labels_3 = list(df_train['Blond_Hair'])
training_labels_4 = list(df_train['Brown_Hair'])
training_labels_5 = list(df_train['Gray_Hair'])
df_train = pd.DataFrame( {'image_id': training_imgs,'Black_Hair': training_labels_2, 'Blond_Hair' : training_labels_3,
                         'Brown_Hair':training_labels_4,'Gray_Hair':training_labels_5})

In [29]:
df_train.Black_Hair = df_train.Black_Hair.astype(str)
df_train.Brown_Hair = df_train.Brown_Hair.astype(str)
df_train.Blond_Hair = df_train.Blond_Hair.astype(str)
df_train.Gray_Hair = df_train.Gray_Hair.astype(str)

In [30]:
df_train['Hair_color'] = df_train['Black_Hair'] + df_train['Brown_Hair'] + df_train['Blond_Hair'] + df_train['Gray_Hair']

In [31]:
df_train.head()

,image_id,Black_Hair,Blond_Hair,Brown_Hair,Gray_Hair,Hair_color
0,000001.jpg,0,0,1,0,0100
1,000002.jpg,0,0,1,0,0100
2,000006.jpg,0,0,1,0,0100
3,000007.jpg,1,0,0,0,1000
4,000008.jpg,1,0,0,0,1000


In [32]:
##Blond=0
df_train = df_train.replace(to_replace ='0010', value=0)

In [33]:
##Black=2
df_train = df_train.replace(to_replace ='1000', value=1)

In [34]:
##Brown=3
df_train = df_train.replace(to_replace ='0100', value=2)

In [35]:
##Gray=4
df_train = df_train.replace(to_replace ='0001', value=3)

In [84]:
df_train.head(10)

,image_id,Black_Hair,Blond_Hair,Brown_Hair,Gray_Hair,Hair_color
0,000001.jpg,0,0,1,0,2
1,000002.jpg,0,0,1,0,2
2,000006.jpg,0,0,1,0,2
3,000007.jpg,1,0,0,0,1
4,000008.jpg,1,0,0,0,1
5,000011.jpg,1,0,0,0,1
6,000012.jpg,1,0,0,0,1
7,000013.jpg,0,1,0,0,0
8,000014.jpg,1,0,0,0,1
9,000017.jpg,1,0,0,0,1


In [36]:
df_train = df_train.astype({'Black_Hair':'int64','Blond_Hair':'int64','Brown_Hair':'int64','Gray_Hair':'int64','Hair_color':'int64'})

In [37]:
df_train.to_csv('df_train_attr.csv',index = False)

In [39]:
df_test = pd.read_csv('output_test.csv')

In [40]:
testing_imgs = ["{}".format(x) for x in list(df_test.image_id)]

In [41]:
testing_labels_1 = list(df_test['image_id'])
testing_labels_2 = list(df_test['Black_Hair'])
testing_labels_3 = list(df_test['Blond_Hair'])
testing_labels_4 = list(df_test['Brown_Hair'])
testing_labels_5 = list(df_test['Gray_Hair'])
df_test = pd.DataFrame( {'image_id': testing_imgs,'Black_Hair': testing_labels_2, 'Blond_Hair' : testing_labels_3,
                         'Brown_Hair':testing_labels_4,'Gray_Hair':testing_labels_5})

In [42]:
df_test.Black_Hair = df_test.Black_Hair.astype(str)
df_test.Brown_Hair = df_test.Brown_Hair.astype(str)
df_test.Blond_Hair = df_test.Blond_Hair.astype(str)
df_test.Gray_Hair = df_test.Gray_Hair.astype(str)

In [43]:
df_test['Hair_color'] = df_test['Black_Hair'] + df_test['Brown_Hair'] + df_test['Blond_Hair'] + df_test['Gray_Hair']

In [44]:
df_test.head()

,image_id,Black_Hair,Blond_Hair,Brown_Hair,Gray_Hair,Hair_color
0,182639.jpg,1,0,0,0,1000
1,182640.jpg,0,0,1,0,0100
2,182642.jpg,1,0,0,0,1000
3,182643.jpg,1,0,0,0,1000
4,182644.jpg,1,0,0,0,1000


In [45]:
##Blond=0
df_test = df_test.replace(to_replace ='0010', value=0)
##Black=2
df_test = df_test.replace(to_replace ='1000', value=1)
##Brown=3
df_test = df_test.replace(to_replace ='0100', value=2)
##Gray=4
df_test = df_test.replace(to_replace ='0001', value=3)

In [46]:
df_test.head()

,image_id,Black_Hair,Blond_Hair,Brown_Hair,Gray_Hair,Hair_color
0,182639.jpg,1,0,0,0,1
1,182640.jpg,0,0,1,0,2
2,182642.jpg,1,0,0,0,1
3,182643.jpg,1,0,0,0,1
4,182644.jpg,1,0,0,0,1


In [47]:
df_test = df_test.astype({'Black_Hair':'int64','Blond_Hair':'int64','Brown_Hair':'int64','Gray_Hair':'int64','Hair_color':'int64'})

In [48]:
df_test.to_csv('df_test_attr.csv',index = False)

In [49]:
df_valid = pd.read_csv('output_valid.csv')

In [50]:
validation_imgs = ["{}".format(x) for x in list(df_test.image_id)]

In [51]:
df_valid.Black_Hair = df_valid.Black_Hair.astype(str)
df_valid.Brown_Hair = df_valid.Brown_Hair.astype(str)
df_valid.Blond_Hair = df_valid.Blond_Hair.astype(str)
df_valid.Gray_Hair = df_valid.Gray_Hair.astype(str)

In [60]:
df_valid['Hair_color'] = df_valid['Black_Hair'] + df_valid['Brown_Hair'] + df_valid['Blond_Hair'] + df_valid['Gray_Hair']

In [61]:
df_valid.head()

,image_id,Black_Hair,Blond_Hair,Brown_Hair,Gray_Hair,Hair_color
0,162772.jpg,1,0,0,0,1000
1,162773.jpg,0,0,1,0,0100
2,162774.jpg,0,0,1,0,0100
3,162775.jpg,1,0,0,0,1000
4,162776.jpg,1,0,0,0,1000


In [62]:
##Brown=0
df_valid = df_valid.replace(to_replace ='0010', value=0)
##Black=1
df_valid = df_valid.replace(to_replace ='1000', value=1)
##Blond=3
df_valid = df_valid.replace(to_replace ='0100', value=2)
##Gray=4
df_valid = df_valid.replace(to_replace ='0001', value=3)

In [63]:
df_valid.head()

,image_id,Black_Hair,Blond_Hair,Brown_Hair,Gray_Hair,Hair_color
0,162772.jpg,1,0,0,0,1
1,162773.jpg,0,0,1,0,2
2,162774.jpg,0,0,1,0,2
3,162775.jpg,1,0,0,0,1
4,162776.jpg,1,0,0,0,1


In [64]:
df_valid = df_valid.astype({'Black_Hair':'int64','Blond_Hair':'int64','Brown_Hair':'int64','Gray_Hair':'int64','Hair_color':'int64'})

In [65]:
df_valid.to_csv('df_valid_attr.csv',index = False)